In [2]:
import numpy as np
import nibabel
import matplotlib.pyplot as plt
from glob import glob
import cv2
import os
import shutil
import statistics
from PIL import Image
import skimage
import logging
logging.basicConfig(level=logging.INFO, force=True)

In [19]:
ZEROS = np.zeros((512, 512))
def liver_and_tumor_png(liver, mask, liver_root, mask_root):
    liver_base = os.path.basename(liver).replace('.nii.gz', '')
    mask_base  = os.path.basename(mask).replace('.nii.gz', '')

    liver_folder_name = os.path.join(liver_root.replace('.nii.gz', ''), liver_base)
    if os.path.exists(liver_folder_name):
        shutil.rmtree(liver_folder_name)
    os.mkdir(liver_folder_name)
    
    mask_folder_name = os.path.join(mask_root.replace('.nii.gz', ''), mask_base)
    if os.path.exists(mask_folder_name):
        shutil.rmtree(mask_folder_name)
    os.mkdir(mask_folder_name)

    liver_img = nibabel.load(liver).get_fdata()
    mask_img2 = nibabel.load(mask).get_fdata()

    liver_img = np.clip(liver_img, -45, 105)
    mask_img = np.clip(mask_img2, 0, 1)
    tumor_img = (mask_img2 > 1).astype(np.uint8) * 255
    
    for d in range(0, liver_img.shape[2]):
        liver_png_file_name = os.path.join(liver_folder_name, f"{d}.png")
        tumor_png_file_name = os.path.join(mask_folder_name, f"{d}.png")
        
        windowed_slice = np.uint8(skimage.exposure.rescale_intensity(
            liver_img[:, :, d], in_range='image', out_range=(0, 255)
        ))

        windowed_mask = np.uint8(skimage.exposure.rescale_intensity(
            mask_img[:, :, d], in_range='image', out_range=(0, 255)
        ))
    
        tumor_mask = tumor_img[..., d]
        final_img = windowed_slice & windowed_mask
    
        if np.all(final_img == ZEROS):
            continue

        skimage.io.imsave(liver_png_file_name, final_img, check_contrast=False)
        skimage.io.imsave(tumor_png_file_name, tumor_mask, check_contrast=False)

In [20]:
volume = './volume-100.nii.gz'
mask = './segmentation-100.nii.gz'
masks = './masks_tumor_png/'
images = './images_tumor_png/'
prediction = './prediction_masks/'

In [21]:
liver_and_tumor_png(volume, mask, './images_tumor_png/', './masks_tumor_png/')